# Classification de documents : prise en main des outils

Le but de ce TP est de classer des documents textuels... Dans un premier temps, nous allons vérifier le bon fonctionnement des outils sur des données jouets puis appliquer les concepts sur des données réelles.


## Conception de la chaine de traitement
Pour rappel, une chaine de traitement de documents classique est composée des étapes suivantes:
1. Lecture des données et importation
    - Dans le cadre de nos TP, nous faisons l'hypothèse que le corpus tient en mémoire... Si ce n'est pas le cas, il faut alors ajouter des structures de données avec des buffers (*data-reader*), bien plus complexes à mettre en place.
    - Le plus grand piège concerne l'encodage des données. Dans le TP... Pas (ou peu) de problème. Dans la vraie vie: il faut faire attention à toujours maitriser les formats d'entrée et de sortie.
1. Traitement des données brutes paramétrique. Chaque traitement doit être activable ou desactivable + paramétrable si besoin.
    - Enlever les informations *inutiles* : chiffre, ponctuations, majuscules, etc... <BR>
    **L'utilité dépend de l'application!**
    - Segmenter en mots (=*Tokenization*)
    - Elimination des stop-words
    - Stemming/lemmatisation (racinisation)
    - Byte-pair encoding pour trouver les mots composés (e.g. Sorbonne Université, Ville de Paris, Premier Ministre, etc...)
1. Traitement des données numériques
    - Normalisation *term-frequency* / binarisation
    - Normalisation *inverse document frequency*
    - Elimination des mots rares, des mots trop fréquents
    - Construction de critère de séparabilité pour éliminer des mots etc...
1. Apprentissage d'un classifieur
    - Choix du type de classifieur
    - Réglage des paramètres du classifieur (régularisation, etc...)

## Exploitation de la chaine de traitement

On appelle cette étape la réalisation d'une campagne d'expériences: c'est le point clé que nous voulons traviller en TAL cette année.
1. Il est impossible de tester toutes les combinaisons par rapport aux propositions ci-dessus... Il faut donc en éliminer un certain nombre.
    - En discutant avec les experts métiers
    - En faisant des tests préliminaires
1. Après ce premier filtrage, il faut:
    - Choisir une évaluation fiable et pas trop lente (validation croisée, leave-one-out, split apprentissage/test simple)
    - Lancer des expériences en grand
        - = *grid-search*
        - parallèliser sur plusieurs machines
        - savoir lancer sur un serveur et se déconnecter
1. Collecter et analyser les résultats


## Inférence

L'inférence est ensuite très classique: la chaine de traitement optimale est apte à traiter de nouveaux documents

# Etape 1: charger les données

In [96]:
import numpy as np
import matplotlib.pyplot as plt

import codecs
import re
import os.path

In [97]:
# Chargement des données:
def load_pres(fname):
    alltxts = []
    alllabs = []
    s=codecs.open(fname, 'r','utf-8') # pour régler le codage
    while True:
        txt = s.readline()
        if(len(txt))<5:
            break
        #
        lab = re.sub(r"<[0-9]*:[0-9]*:(.)>.*","\\1",txt)
        txt = re.sub(r"<[0-9]*:[0-9]*:.>(.*)","\\1",txt)
        if lab.count('M') >0:
            alllabs.append(-1)
        else: 
            alllabs.append(1)
        alltxts.append(txt)
    return alltxts,alllabs


In [98]:
import string
import unicodedata
import nltk
from   nltk.stem    import WordNetLemmatizer
from nltk.stem.snowball import FrenchStemmer

class preprocessing():
    """
    All functions related to preprocessing
    """
    
    def __init__(self) -> None:   
        pass

    def do_all(self, text : tuple, my_punc : string = '\n\r\t') -> tuple:
        """
        Apply all the preprocessing function in one loop (to save time) (Except lemmatisation)
        """
        punc = string.punctuation  
        punc += my_punc
        return [(unicodedata.normalize('NFD'
                , re.sub('[0-9]+', ''
                , line.lower()
                .translate(str.maketrans(punc, ' ' * len(punc)))))
                .encode('ascii', 'ignore').decode("utf-8")
                .replace("  ", " ")
                .strip()
                ,pol) for line,pol in text]
    
    def remove_maj(self, text : tuple) -> tuple :
        """
        Return a tuple of text in lower case
        """
        return [(line.lower(),pol) for line,pol in text]

    def remove_punctuation(self, text  : tuple, my_punc : string = '\n\r\t') -> tuple :
        """
        Return a tuple of text without punctuation
        """
        punc = string.punctuation  
        punc += my_punc
        return [(line.translate(str.maketrans(punc, ' ' * len(punc))),pol) for line, pol in text]

    def remove_numbers(self, text : tuple) -> tuple :
        """
        Return a tuple of text without numbers
        """
        return [(re.sub('[0-9]+', '', line),pol) for line,pol in text]

    def remove_non_normalized_char(self, text : tuple) -> tuple :
        """
        Return a tuple of text without non normalized char
        """
        return [(unicodedata.normalize('NFD', line).encode('ascii', 'ignore').decode("utf-8"),pol) for line,pol in text]

    def get_line(self, text : tuple, sep : str = '\n', n : int = 0) -> tuple :
        """
        Returns a text list with the n-th line 
            - sep : string to recognize a new line
            - n : Integer of the line (0 = first, -1 = last)
        """
        return [(line.strip().split(sep)[n], pol) for line,pol in text]

    def remove_space(self, text : tuple) -> tuple:
        """
        Return a tuple of text without supernumerary space
        """
        return  [(line.replace("  ", " ").strip(), pol) for line,pol in text]

    def lemmatisation(self, text : tuple) -> tuple :
        """
        Return a lemmatized list 
        """
        # Téléchargez le stemmer français
        nltk.download('stopwords')
        nltk.download('punkt')
        nltk.download('rslp')

        # Créez un objet stemmer français
        stemmer = FrenchStemmer()
        l_words = [(nltk.word_tokenize(line),pol) for line,pol in text]
        stemmed_words = [([stemmer.stem(word) for word in line],pol) for line,pol in l_words]
        # print(stemmed_words)
        return [(" ".join(line),pol) for line,pol in stemmed_words]


In [99]:
president_path_train = "./ressources/AFDpresidentutf8/corpus.tache1.learn.utf8"
president_path_test = "./ressources/AFDpresidentutf8/corpus.tache1.test.utf8"

# Parsing
palltxts_train, palllabs_train = load_pres(president_path_train)
palltxts_test, palllabs_test = load_pres(president_path_test)

# Zip in tupe
p_train = list(zip(palltxts_train, palllabs_train))
p_test = list(zip(palltxts_test, palllabs_test))

In [100]:
def load_movies(path2data): # 1 classe par répertoire
    alltxts = [] # init vide
    labs = []
    cpt = 0
    for cl in os.listdir(path2data): # parcours des fichiers d'un répertoire
        for f in os.listdir(path2data+cl):
            txt = open(path2data+cl+'/'+f).read()
            alltxts.append(txt)
            labs.append(cpt)
        cpt+=1 # chg répertoire = cht classe
        
    return alltxts,labs


In [101]:
# Faire la même pour les movies
# TODO

In [102]:
# print(len(alltxts),len(alllabs))
# print(alltxts[0])
# print(alllabs[0])
# print(alltxts[-1])
# print(alllabs[-1])

# Transformation paramétrique du texte

Vous devez tester, par exemple, les cas suivants:
- transformation en minuscule ou pas
- suppression de la ponctuation
- transformation des mots entièrement en majuscule en marqueurs spécifiques
- suppression des chiffres ou pas
- conservation d'une partie du texte seulement (seulement la première ligne = titre, seulement la dernière ligne = résumé, ...)
- stemming
- ...


Vérifier systématiquement sur un exemple ou deux le bon fonctionnement des méthodes sur deux documents (au moins un de chaque classe).

In [103]:
# Testing preprocessing functions

preprocessor = preprocessing()

# President
two_line = p_train[0:2]
two_line[0] = ('   999' + two_line[0][0], two_line[0][1])
print("Normal text :")
print(two_line)

print("\nMinuscule :")
print(preprocessor.remove_maj(two_line))

print("\nNumbers :")
print(preprocessor.remove_numbers(two_line))

print("\nNormalized char :")
print(preprocessor.remove_non_normalized_char(two_line))

print("\npunctuation :")
print(preprocessor.remove_punctuation(two_line))

print("\nSpace :")
print(preprocessor.remove_space(two_line))

print("\nLemmatisation :")
print(preprocessor.lemmatisation(two_line))

print("\nGet line :") # Here as the separator use is a space it will get the last word
print(preprocessor.get_line(two_line, sep = ' ', n = -1))

print("\nAll :")
print(preprocessor.do_all(two_line))

Normal text :
[("   999 Quand je dis chers amis, il ne s'agit pas là d'une formule diplomatique, mais de l'expression de ce que je ressens.\r\n", 1), (" D'abord merci de cet exceptionnel accueil que les Congolais, les Brazavillois, nous ont réservé cet après-midi.\r\n", 1)]

Minuscule :
[("   999 quand je dis chers amis, il ne s'agit pas là d'une formule diplomatique, mais de l'expression de ce que je ressens.\r\n", 1), (" d'abord merci de cet exceptionnel accueil que les congolais, les brazavillois, nous ont réservé cet après-midi.\r\n", 1)]

Numbers :
[("    Quand je dis chers amis, il ne s'agit pas là d'une formule diplomatique, mais de l'expression de ce que je ressens.\r\n", 1), (" D'abord merci de cet exceptionnel accueil que les Congolais, les Brazavillois, nous ont réservé cet après-midi.\r\n", 1)]

Normalized char :
[("   999 Quand je dis chers amis, il ne s'agit pas la d'une formule diplomatique, mais de l'expression de ce que je ressens.\r\n", 1), (" D'abord merci de cet exc

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Ju'\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Ju'\AppData\Roaming\nltk_data...


[("999 quand je dis cher amis , il ne s'ag pas là d'un formul diplomat , mais de l'express de ce que je ressen .", 1), ("d'abord merc de cet exceptionnel accueil que le congol , le brazavillois , nous ont réserv cet après-mid .", 1)]

Get line :
[('ressens.', 1), ('après-midi.', 1)]

All :
[('quand je dis chers amis il ne s agit pas la d une formule diplomatique mais de l expression de ce que je ressens', 1), ('d abord merci de cet exceptionnel accueil que les congolais les brazavillois nous ont reserve cet apres midi', 1)]


[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package rslp to
[nltk_data]     C:\Users\Ju'\AppData\Roaming\nltk_data...
[nltk_data]   Package rslp is already up-to-date!


In [104]:
# Preprocessing Text

# President 
## Train
p_train = preprocessor.do_all(p_train)
# p_train = preprocessor.lemmatisation(p_train)

## Test
p_test = preprocessor.do_all(p_test)
# p_test = preprocessor.lemmatisation(p_test)

# Movie
# TODO

# Extraction du vocabulaire

Exploration préliminaire des jeux de données.

- Quelle est la taille d'origine du vocabulaire?
- Que reste-t-il si on ne garde que les 100 mots les plus fréquents? [word cloud]
- Quels sont les 100 mots dont la fréquence documentaire est la plus grande? [word cloud]
- Quels sont les 100 mots les plus discriminants au sens de odds ratio? [word cloud]
- Quelle est la distribution d'apparition des mots (Zipf)
- Quels sont les 100 bigrammes/trigrammes les plus fréquents?


In [106]:
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import chi2
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from collections import Counter
import math
from scipy import special


class voc_extracting():

    def __init__(self) -> None:
        self.vectorizer = CountVectorizer()


    def get_size(self, text : tuple) -> int :
        """
        Return the size of a vocabulary
        """
        corpus = [line for line,pol in text]
        self.vectorizer.fit_transform(corpus)
        return len(self.vectorizer.get_feature_names())

    def get_zipf_distribution(self, text : tuple, n : int = 100) -> list :
        vectorizer = CountVectorizer()
        corpus = [line for line,pol in text]
        X = vectorizer.fit_transform(corpus)
        sum_words = X.sum(axis=0) 
        words_freq = [(word, sum_words[0, idx]) for word, idx in vectorizer.vocabulary_.items()]
        words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
        max_f = words_freq[0][1]
        a = 2. #  distribution parameter
        return [w for w,f in words_freq[:n]]

    def get_tf_idf(self, text : tuple, n : int = 100) -> list :
        """
        Return the list of n words the most frequent according to TF-IDF
        """
        # self.get_most_freq_word(text, n)
        # tot_word = sum([f for w,f in self.wc.most_common() if len(w) > 1])
        # tf_list = [(w,f/tot_word) for w,f in self.wc.most_common() if len(w) > 1]
        # tf_list = sorted(tf_list, key=lambda x: x[0], reverse=False)

        corpus = [line for line,pol in text]
        vectorizer = TfidfVectorizer()
        X = vectorizer.fit_transform(corpus)
        # feature_names = vectorizer.get_feature_names()
        # nb_doc = len(text)
        # IDF = np.asarray(nb_doc/X.sum(axis=0)).ravel()
        # IDF_log = [math.log(x) for x in IDF]
        # scores = [(tf[0],tf[1]-idf) for tf,idf in zip(tf_list,IDF_log)]


        scores = zip(vectorizer.get_feature_names(),
                 np.asarray(X.sum(axis=0)).ravel())  # donne le même résultat que la formule tf-idf

        sorted_scores = sorted(scores, key=lambda x: x[1], reverse=True)
        return [w[0] for w in sorted_scores[:n]]
    
    def get_most_freq_word(self, text : tuple, n : int = 100) -> list :
        """
        Return the list of n words the most frequent
        """
        vectorizer = CountVectorizer()
        corpus = [line for line,pol in text]
        X = vectorizer.fit_transform(corpus)
        sum_words = X.sum(axis=0) 
        words_freq = [(word, sum_words[0, idx]) for word, idx in vectorizer.vocabulary_.items()]
        words_freq = sorted(words_freq, key = lambda x: x[1], reverse=True)
        return [w for w,f in words_freq[:n]]

    def word_cloud(self, text : tuple, stop_word : list = None) -> None :
        """
        Display word cloud
        """
        corpus = [line for line,pol in text]
        corpus = ' '.join(corpus)
        wordcloud = WordCloud(stopwords = stop_word).generate(corpus)
        plt.imshow(wordcloud, interpolation="bilinear")
        WordCloud()

    def get_odd(self, text : tuple, n : int = 100) -> list :
        corpus = [ line for line,pol in text]
        vectorizer = CountVectorizer()
        X = vectorizer.fit_transform(corpus)
        features = vectorizer.get_feature_names()
        chi2score = chi2(X, y)[0]
        # scores = list(zip(features, chi2score))


        # oddsratios, pvals = list(map(lambda x: list(x), zip(*[f_classif(X.toarray(), y) for y in [0,1]])))
        # sorted_oddsratios = sorted(zip(X_features, oddsratios, pvals), key=lambda x: x[1], reverse=True)
        # return sorted_oddsratios[:n]


In [107]:
def get_odd_ratios_discr_words(self, text : tuple):
    """
    Return a list of discriminant words
    """
    vectorizer = CountVectorizer()
    corpus = [line for line,pol in text]
    X = vectorizer.fit_transform(corpus)

    pos1 = [i for i, (line, pol) in enumerate(text) if pol == 1]
    pos2 = [i for i, (line, pol) in enumerate(text) if pol == -1]
    X_class1 = X[pos1,:]
    X_class2 = X[pos2,:]

    sum_words1 = X_class1.sum(axis=0)
    sum_words2 = X_class2.sum(axis=0)

    n_voc = len(vectorizer.vocabulary_)
    words_freq1 = [(word, sum_words1[0, idx]) for word, idx in vectorizer.vocabulary_.items()]
    words_freq2 = [(word, sum_words2[0, idx]) for word, idx in vectorizer.vocabulary_.items()]


    odd_ratios = [(word, get_odd_ratio(words_freq1[idx][1]/n_voc, words_freq2[idx][1]/n_voc)) for word, idx in vectorizer.vocabulary_.items()]
       
    odd_ratios = sorted(odd_ratios, key = lambda x: x[1], reverse=True)
    return [w for w,f in words_freq[:n]]
    

In [172]:
def get_odd_ratio(p : float, q : float):
    """
    Calculate odd ratios from the frequencies p and q
    """
    if p not in [0,1] and q not in [0,1]:
        OR = (p*(1-q)) / (q*(1-p))
        #OR = np.log(p*(1-q)) - np.log(q*(1-p))
        return OR
    else:
        return 0 #A changer peut etre

In [229]:
vectorizer = CountVectorizer()
corpus = [line for line,pol in p_train]
X = vectorizer.fit_transform(corpus)

#transform

pos1 = [i for i, (line, pol) in enumerate(p_train) if pol == 1]
pos2 = [i for i, (line, pol) in enumerate(p_train) if pol == -1]
#X_class1 = X[pos1,:]
#X_class2 = X[pos2,:]
corpus1 = [line for line,pol in [p_train[p1] for p1 in pos1]]
corpus2 = [line for line,pol in [p_train[p1] for p1 in pos2]]
X_class1 = vectorizer.transform(corpus1)
X_class2 = vectorizer.transform(corpus2)

print(X_class1.shape)
print(X_class2.shape)

sum_words1 = X_class1.sum(axis=0)
sum_words2 = X_class2.sum(axis=0)

n_voc = len(vectorizer.vocabulary_)
words_freq1 = [(word, sum_words1[0, idx]) for word, idx in vectorizer.vocabulary_.items()]
words_freq2 = [(word, sum_words2[0, idx]) for word, idx in vectorizer.vocabulary_.items()]


odd_ratios = [(word, get_odd_ratio(words_freq1[idx][1]/n_voc, words_freq2[idx][1]/n_voc)) for word, idx in vectorizer.vocabulary_.items()]
odd_ratios = sorted(odd_ratios, key = lambda x: x[1], reverse=True)
odd_ratios

(49890, 27054)
(7523, 27054)


[('accueillie', 359.70691786209227),
 ('assouplie', 192.35092878680712),
 ('blindee', 163.98196422594918),
 ('ajustements', 146.78675486844062),
 ('attendiez', 136.68207147633552),
 ('mondiales', 104.39747680890538),
 ('changeraient', 97.18618070808978),
 ('evoquaient', 93.31734727940359),
 ('achetant', 88.28391307572498),
 ('antillo', 88.06317199300568),
 ('arawaks', 79.68933885222062),
 ('connaissant', 79.45820939916717),
 ('clair', 76.21128326784788),
 ('anterieure', 75.20571555654398),
 ('dan', 75.20571555654398),
 ('anormale', 73.39066448639808),
 ('anticipees', 73.19480375078759),
 ('agrandi', 72.23275829948956),
 ('appropriees', 71.36935196195004),
 ('cantons', 71.18419004558424),
 ('alternative', 70.358921007654),
 ('battus', 70.17899495997627),
 ('bergere', 69.17387437465258),
 ('connexion', 68.16882828133106),
 ('agents', 67.66034495632047),
 ('adaptabilite', 66.15895953757226),
 ('acharnement', 64.8042339832869),
 ('antenne', 64.299487484216),
 ('aggrave', 63.14471490496832)

In [ ]:
def get_doc_freq_word(self, text : tuple, n : int = 100) -> list :
    """
    Return the list of n words the most frequent
    """
    vectorizer = CountVectorizer()
    corpus = [line for line,pol in text]
    X = vectorizer.fit_transform(corpus)
    words_freq = [(word, (np.transpose(X)[idx]!=0).sum(axis=1)[0,0]) for word, idx in vectorizer.vocabulary_.items()]
    words_freq = sorted(words_freq, key = lambda x: x[1], reverse=True)
    return [w for w,f in words_freq[:n]]

In [225]:
vectorizer = CountVectorizer()
corpus = [line for line,pol in p_train]
X = vectorizer.fit_transform(corpus)
words_freq = [(word, (np.transpose(X)[idx]!=0).sum(axis=1)[0,0]) for word, idx in vectorizer.vocabulary_.items()]
words_freq = sorted(words_freq, key = lambda x: x[1], reverse=True)
words_freq #lent

[('de', 35268),
 ('la', 27664),
 ('et', 26298),
 ('le', 19625),
 ('les', 18229),
 ('des', 16088),
 ('est', 14927),
 ('en', 13349),
 ('que', 13140),
 ('qui', 12395),
 ('un', 10791),
 ('une', 10692),
 ('dans', 10113),
 ('pour', 9892),
 ('du', 8966),
 ('il', 8861),
 ('je', 8808),
 ('nous', 7696),
 ('au', 7269),
 ('vous', 7140),
 ('ce', 6990),
 ('plus', 5882),
 ('pas', 5704),
 ('qu', 5551),
 ('sur', 5223),
 ('notre', 5063),
 ('france', 4937),
 ('ne', 4890),
 ('par', 4849),
 ('cette', 4558),
 ('mais', 4388),
 ('avec', 4345),
 ('aussi', 4232),
 ('se', 4163),
 ('nos', 4138),
 ('pays', 4126),
 ('ou', 4122),
 ('sont', 4104),
 ('elle', 3848),
 ('ont', 3641),
 ('aux', 3615),
 ('etre', 3608),
 ('tout', 3453),
 ('tous', 3247),
 ('votre', 3059),
 ('leur', 2994),
 ('son', 2834),
 ('bien', 2821),
 ('nom', 2755),
 ('ces', 2712),
 ('meme', 2692),
 ('comme', 2642),
 ('hui', 2558),
 ('aujourd', 2557),
 ('on', 2497),
 ('entre', 2476),
 ('doit', 2389),
 ('europe', 2382),
 ('monde', 2365),
 ('faire', 2327),


Question qui devient de plus en plus intéressante avec les approches modernes:
est-il possible d'extraire des tri-grammes de lettres pour représenter nos documents?

Quelle performances attendrent? Quels sont les avantages et les inconvénients d'une telle approche?

In [14]:
extractor = voc_extracting()

# President
print("Size Voc :")
print(extractor.get_size(p_train))

# print("World Cloud :")
# print(extractor.word_cloud(p_train))

print("Most frequent words (100)")
print(extractor.get_most_freq_word(p_train, 100)) # Beaucoup de stop word, premier mot intéressant à l'index 35 -> france

print("TF-IDF (100) :")
print(extractor.get_tf_idf(p_train, 100)) # le mot france est mieux classé, diminution de l'influence des stop word

# print("ODD (100) :")
# print(extractor.get_odd(p_train, 100))

# print("Zipf (100) :")
# print(extractor.get_zipf_distribution(p_train, 100)) # le mot france est mieux classé mais moins bien que TF-IFD

Size Voc :


C:\Anaconda\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


27054
Most frequent words (100)
['de', 'la', 'et', 'le', 'les', 'des', 'est', 'en', 'que', 'qui', 'un', 'une', 'pour', 'dans', 'du', 'je', 'il', 'nous', 'vous', 'au', 'ce', 'plus', 'qu', 'pas', 'sur', 'notre', 'par', 'ne', 'france', 'nos', 'avec', 'cette', 'ou', 'se', 'mais', 'pays', 'sont', 'elle', 'aussi', 'aux', 'ont', 'etre', 'leur', 'tout', 'votre', 'nom', 'tous', 'son', 'on', 'bien', 'ces', 'meme', 'ses', 'comme', 'entre', 'europe', 'sa', 'hui', 'aujourd', 'monde', 'doit', 'faire', 'francais', 'ai', 'ils', 'si', 'faut', 'sans', 'ete', 'fait', 'date', 'etat', 'leurs', 'cela', 'avez', 'dire', 'tres', 'deux', 'ensemble', 'peut', 'developpement', 'dont', 'vos', 'autres', 'president', 'politique', 'monsieur', 'encore', 'toutes', 'vie', 'avons', 'ceux', 'temps', 'ici', 'depuis', 'toute', 'paix', 'union', 'chacun', 'avenir']
TF-IDF (100) :
['de', 'la', 'et', 'le', 'les', 'des', 'est', 'en', 'que', 'qui', 'une', 'un', 'pour', 'dans', 'il', 'nous', 'je', 'vous', 'du', 'ce', 'plus', 'au', 

# Modèles de Machine Learning

Avant de lancer de grandes expériences, il faut se construire une base de travail solide en étudiant les questions suivantes:

- Combien de temps ça prend d'apprendre un classifieur NB/SVM/RegLog sur ces données en fonction de la taille du vocabulaire?
- La validation croisée est-elle nécessaire? Est ce qu'on obtient les mêmes résultats avec un simple *split*?
- La validation croisée est-elle stable? A partir de combien de fold (travailler avec différentes graines aléatoires et faire des statistiques basiques)?

## Première campagne d'expériences

Les techniques sur lesquelles nous travaillons étant sujettes au sur-apprentissage: trouver le paramètre de régularisation dans la documentation et optimiser ce paramètre au sens de la métrique qui vous semble la plus appropriée (cf question précédente).

## Equilibrage des données

Un problème reconnu comme dur dans la communauté est celui de l'équilibrage des classes (*balance* en anglais). Que faire si les données sont à 80, 90 ou 99% dans une des classes?
Le problème est dur mais fréquent; les solutions sont multiples mais on peut isoler 3 grandes familles de solution.

1. Ré-équilibrer le jeu de données: supprimer des données dans la classe majoritaire et/ou sur-échantilloner la classe minoritaire.<BR>
   $\Rightarrow$ A vous de jouer pour cette technique
1. Changer la formulation de la fonction de coût pour pénaliser plus les erreurs dans la classe minoritaire:
soit une fonction $\Delta$ mesurant les écarts entre $f(x_i)$ et $y_i$ 
$$C = \sum_i  \alpha_i \Delta(f(x_i),y_i), \qquad \alpha_i = \left\{
\begin{array}{ll}
1 & \text{si } y_i \in \text{classe majoritaire}\\
B>1 & \text{si } y_i \in \text{classe minoritaire}\\
\end{array} \right.$$
<BR>
   $\Rightarrow$ Les SVM et d'autres approches sklearn possèdent des arguments pour régler $B$ ou $1/B$... Ces arguments sont utiles mais pas toujours suffisant.
1. Courbe ROC et modification du biais. Une fois la fonction $\hat y = f(x)$ apprise, il est possible de la *bidouiller* a posteriori: si toutes les prédictions $\hat y$ sont dans une classe, on va introduire $b$ dans $\hat y = f(x) + b$ et le faire varier jusqu'à ce qu'un des points change de classe. On peut ensuite aller de plus en plus loin.
Le calcul de l'ensemble des scores associés à cette approche mène directement à la courbe ROC.

**Note:** certains classifieurs sont intrinsèquement plus résistante au problème d'équilibrage, c'est par exemple le cas des techniques de gradient boosting que vous verrez l'an prochain.